In [1]:
import sys
sys.path.append('../')
from setting import config_read

In [2]:
from owlready2 import *
import pandas as pd
from elasticsearch import Elasticsearch

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [3]:
# Read config file
config = config_read('../')

# Load owl file
data_path = config['owl']['path']
onto = get_ontology(data_path).load()

In [4]:
# Elasticsearch
server_ip = config['elasticsearch']['ip']
index_name = config['elasticsearch']['name']
es = Elasticsearch(server_ip)

In [5]:
def change_prefix(s):
    s = str(s)
    s_list = s.rsplit('.',1) 

    if s_list[0] == config['owl']['path'][:-4]:
        s = 'skmo:' + s_list[-1]
    else:
        s = s_list[0] + ':' + s_list[-1]
    return s

In [6]:
index_list = []
index_list.extend([(change_prefix(r), "T_c", r.label) for r in onto.classes()])
index_list.extend([(change_prefix(r), "T_op", r.label) for r in onto.object_properties()])
index_list.extend([(change_prefix(r), "T_dp", r.label) for r in onto.data_properties()])

In [7]:
instance_list = []
for r in onto.individuals():
    if r.is_a[0] == Thing:  continue
    tbox = [change_prefix(t) for t in r.is_a]
    instance_list.append((change_prefix(r), "T_i", r.label, tbox))

In [9]:
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name,ignore=[400, 404])
    print('Index has been deleted successfully')

Index has been deleted successfully


In [10]:
for row in index_list:
    doc = {'URI':row[0], 'Type':row[1], 'Annotation Values':row[2]}
    es.index(index=index_name, body=doc)

In [11]:
for row in instance_list:
    doc = {'URI':row[0], 'Type':row[1], 'Annotation Values':row[2], 'Tbox':row[3]}
    es.index(index=index_name, body=doc)